<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/DEEPOSEEK_GLM_AGENTIC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain-core langgraph langchain-zhipuai langchain-openai -q

In [ ]:
!pip install ccxt -q

In [ ]:
!pip install --upgrade langchain-zhipuai zhipuai -q

In [ ]:
# 1. Uninstall the extremely old version
!pip uninstall -y langchain-zhipuai

# 2. Install the latest version of the ZhipuAI integration package.
# The correct, modern package name is often just 'langchain-zhipuai' or 'langchain-community'.
# We assume the modern package structure is correct now.
!pip install --upgrade langchain-zhipuai

In [ ]:
!pip show langchain-zhipuai

Name: langchain-zhipuai
Version: 0.0.1
Summary: LangChain interface for Zhipu AI
Home-page: https://pypi.org/project/langchain-zhipuai
Author: Yi ZHANG
Author-email: yizhang.dev@gmail.com
License: BSD-3-Clause
Location: /usr/local/lib/python3.12/dist-packages
Requires: 
Required-by: 


In [ ]:
import operator
import os
import logging
import ccxt
from typing import TypedDict, Annotated, List
from langchain_core.messages import BaseMessage, HumanMessage, SystemMessage, AIMessage
from langgraph.graph import StateGraph, END
from langchain_openai import ChatOpenAI
from openai import OpenAI # The native client for GLM (Zhipu AI)

# Set up basic logging
logger = logging.getLogger(__name__)

# --- 2. Environment and LLM Initialization ---

glm_client = None
deepseek_llm = None
kraken_exchange = None
GLM_STATUS = "Mock (Key Missing)"

# API Key Loading from Userdata
try:
    from google.colab import userdata
    ZAI_KEY_API = userdata.get('ZAI_KEY')
    DEEPSEEK_API_KEY = userdata.get('DEEPSEEK_API_KEY')
    KRAKEN_API_KEY = userdata.get('KRAKEN')
    KRAKEN_SECRET = userdata.get('KRAKEN_SECRET')
except Exception as e:
    print(f"Error loading Colab Userdata: {e}. All keys initialized as None.")
    ZAI_KEY_API = None
    DEEPSEEK_API_KEY = None
    KRAKEN_API_KEY = None
    KRAKEN_SECRET = None

# Initialize GLM LLM Client (Direct OpenAI Client for Zhipu AI)
if ZAI_KEY_API:
    try:
        # --- GLM INITIALIZATION ---
        glm_client = OpenAI(
            api_key=ZAI_KEY_API,
            # Using the official base URL for OpenAI compatibility
            base_url="https://open.bigmodel.cn/api/paas/v4/"
        )
        GLM_STATUS = "Live"
        print("GLM LLM (ZhipuAI via OpenAI Client) Initialized: LIVE")
    except Exception as e:
        print(f"GLM LLM (ZhipuAI) FAILED TO CONNECT ({e}). Setting to None.")
        glm_client = None
        GLM_STATUS = "Mock (API Failed)"
else:
    print(f"GLM LLM (ZhipuAI) Initialized: {GLM_STATUS}")

# Initialize DeepSeek LLM (LangChain wrapper for OpenAI compatible API)
if DEEPSEEK_API_KEY:
    try:
        deepseek_llm = ChatOpenAI(
            api_key=DEEPSEEK_API_KEY,
            base_url="https://api.deepseek.com/v1",
            model="deepseek-reasoner",
            temperature=0.0
        )
        print("DeepSeek LLM (LangChain) Initialized: LIVE")
    except Exception as e:
        print(f"DeepSeek LLM FAILED TO CONNECT ({e}). Setting to None.")
        deepseek_llm = None
else:
    print("DeepSeek LLM NOT INITIALIZED (Missing DEEPSEEK_API_KEY).")

# Initialize CCXT/Kraken Exchange
if KRAKEN_API_KEY and KRAKEN_SECRET:
    try:
        kraken_exchange = ccxt.kraken({
            'apiKey': KRAKEN_API_KEY,
            'secret': KRAKEN_SECRET,
            'enableRateLimit': True,
        })
        print("Kraken Exchange (CCXT) Initialized: LIVE")
    except Exception as e:
        print(f"Error initializing Kraken CCXT: {e}. Using Mock Data.")
        kraken_exchange = None
else:
    print("Kraken Exchange NOT INITIALIZED (Missing API keys). Using Mock Data.")


# --- 3. Define Tools (Using CCXT for Crypto Data) ---

@tool
def crypto_price_tool(ticker: str) -> str:
    """Fetches the current price and indicators using the Kraken API via CCXT."""
    if not kraken_exchange:
        # Fallback to the original mock data if API initialization failed
        data = {"ETH": {"price": 3500.50, "24h_change": "+4.1%", "high": 3650.00, "low": 3400.00, "trend": "Positive news (Mock)."}}
        ticker_key = ticker.upper()
        if ticker_key in data:
            d = data[ticker_key]
            return (f"MOCK DATA for {ticker_key}/USD. Current Price: ${d['price']:.2f}. "
                    f"24h Change: {d['24h_change']}. 24h Range: ${d['low']:.2f} to ${d['high']:.2f}.")
        else:
            return f"Error: No mock or live data found for {ticker}."

    try:
        symbol = f"{ticker.upper()}/USD"
        ticker_data = kraken_exchange.fetch_ticker(symbol)

        last_price = ticker_data['last']
        percent_change = ticker_data['percentage']
        high_24h = ticker_data['high']
        low_24h = ticker_data['low']

        mid_range = (high_24h + low_24h) / 2
        trend = "Bullish (price closer to 24h high)" if last_price >= mid_range else "Bearish (price closer to 24h low)"

        return (
            f"LIVE KRAKEN DATA for {symbol}: Current Price: ${last_price:.2f}. "
            f"24h Price Change: {percent_change:.2f}%. 24h Range: ${low_24h:.2f} to ${high_24h:.2f}. "
            f"Current Technical Trend: {trend}. Note: Resistance/Support levels derived from range."
        )
    except Exception as e:
        return f"CCXT Error: {e}"


# --- 4. Define Graph State (Shared Memory) ---

class AgentState(TypedDict):
    """The state shared between nodes in LangGraph."""
    query: str
    tool_output: str
    risk_analysis: str
    technical_analysis: str
    final_recommendation: str
    messages: Annotated[List[BaseMessage], operator.add]

# --- 5. Agent Nodes ---

def risk_analyst_glm(state: AgentState) -> AgentState:
    """Node 2: GLM-4.6 (Qualitative/Risk) performs analysis using the native OpenAI Client."""

    if glm_client is None:
        return {"risk_analysis": f"GLM-4.6 MOCK: Client not connected. Cannot perform qualitative analysis. (Status: {GLM_STATUS})"}

    system_prompt = (
        "You are the GLM-4.6 Qualitative Risk Analyst. Analyze the '24h Price Change' and "
        "'Technical Trend' from the provided data. Identify the *implied market sentiment* "
        "and suggest potential *macro risks*. Keep it concise. Data: {data}"
    )
    prompt = system_prompt.format(data=state["tool_output"])

    try:
        # --- DIRECT OPENAI CLIENT CALL (Model ID updated to glm-4.6) ---
        response = glm_client.chat.completions.create(
            model="glm-4.6",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": prompt}
            ],
            temperature=0.1
        )
        # Extract content from the direct SDK response object
        analysis_content = response.choices[0].message.content

    except Exception as e:
        analysis_content = f"GLM-4.6 API FAILED during invocation: {e}"

    return {"risk_analysis": analysis_content, "messages": [AIMessage(content=f"Risk Analyst (GLM) completed: {analysis_content}")]}


def technical_analyst_deepseek(state: AgentState) -> AgentState:
    """Node 3: DeepSeek (Quantitative) performs analysis using LangChain wrapper."""

    if deepseek_llm is None:
        return {"technical_analysis": "DeepSeek Mock: LLM failed to initialize. Cannot perform quantitative analysis."}

    system_prompt = (
        "You are the DeepSeek Quantitative Analyst, specializing in price action. "
        "Analyze the 'Current Price', '24h Range (Low to High)', and '24h Change'. "
        "Based on the range, identify the closest potential breakout (Resistance) and breakdown (Support) points. "
        "Provide a brief *quantitative assessment* of the immediate price outlook. Data: {data}"
    )
    prompt = system_prompt.format(data=state["tool_output"])

    # LangChain invocation
    response = deepseek_llm.invoke([HumanMessage(content=prompt)])

    return {"technical_analysis": response.content, "messages": [AIMessage(content=f"Technical Analyst (DeepSeek) completed: {response.content}")]}

def data_fetcher(state: AgentState) -> AgentState:
    """Node 1: Fetches the raw data."""
    query = state["query"]
    ticker = "ETH" if "ethereum" in query.lower() or "eth" in query.lower() else "BTC"
    data = crypto_price_tool.invoke({"ticker": ticker})
    return {"tool_output": data, "messages": [AIMessage(content=f"Data Fetcher completed. Raw data: {data}")]}

def final_synthesizer(state: AgentState) -> AgentState:
    """Node 4: Synthesizes both analyses into a final recommendation."""

    llm_for_synthesis = deepseek_llm

    if llm_for_synthesis is None:
        # Fallback using GLM client if DeepSeek failed
        if glm_client:
            analysis_content = f"DeepSeek failed. Synthesizing analysis using GLM: \n{state['risk_analysis']}\n{state['technical_analysis']}"
            try:
                # Use GLM for fallback synthesis
                response = glm_client.chat.completions.create(
                    model="glm-4", messages=[{"role": "user", "content": analysis_content}]
                )
                return {"final_recommendation": response.choices[0].message.content, "messages": [AIMessage(content="Final Strategist completed via GLM fallback.")]}
            except:
                return {"final_recommendation": "Synthesis Error: Both LLMs failed to initialize."}
        else:
            return {"final_recommendation": "Synthesis Error: Both LLMs failed to initialize."}

    system_prompt = (
        "You are the Lead Investment Strategist. Synthesize the Qualitative Analysis (GLM-4.6) and "
        "Quantitative Analysis (DeepSeek) below to provide a single, actionable investment recommendation (Buy/Hold/Sell) "
        "for the user's query. Justify your answer by balancing risk and technical momentum. "
        "NOTE: If one analysis result contains 'MOCK' or 'FAILED', base the final recommendation primarily on the live analysis, but "
        "state the risk of the missing information."
        f"Original Query: {state['query']}"
    )

    full_analysis = (
        f"--- Qualitative Risk Analysis (GLM-4.6) ---\n{state['risk_analysis']}\n\n"
        f"--- Quantitative Technical Analysis (DeepSeek) ---\n{state['technical_analysis']}"
    )

    # LangChain invocation for the final synthesis (DeepSeek)
    response = llm_for_synthesis.invoke([
        SystemMessage(content=system_prompt),
        HumanMessage(content=full_analysis)
    ])

    return {"final_recommendation": response.content, "messages": [AIMessage(content=f"Final Strategist completed: {response.content}")]}

# --- 6. Define and Run the Workflow Graph (LangGraph) ---

workflow = StateGraph(AgentState)

# Add the nodes
workflow.add_node("data_fetcher", data_fetcher)
workflow.add_node("risk_analyst", risk_analyst_glm)
workflow.add_node("technical_analyst", technical_analyst_deepseek)
workflow.add_node("final_synthesizer", final_synthesizer)

# Define the flow (Edges)
workflow.set_entry_point("data_fetcher")
workflow.add_edge("data_fetcher", "risk_analyst")
workflow.add_edge("data_fetcher", "technical_analyst")

# Fan-in
workflow.add_edge("risk_analyst", "final_synthesizer")
workflow.add_edge("technical_analyst", "final_synthesizer")

# End the graph
workflow.add_edge("final_synthesizer", END)

# Compile the graph
app = workflow.compile()

# --- 7. Execute Example ---

user_query = "What is the current investment thesis for Ethereum (ETH) based on live Kraken data? Provide a Buy/Hold/Sell recommendation."

print(f"\n{'='*80}\nRUNNING MULTI-AGENT CRYPTO ANALYSIS\n{'='*80}")
print(f"Query: {user_query}")
print(f"Agents Status: GLM-4.6 ({GLM_STATUS}), DeepSeek ({'Live' if deepseek_llm else 'Mock'})")
print(f"Data Source: Kraken Exchange (via CCXT)\n")


final_state = app.invoke(
    {
        "query": user_query,
        "tool_output": "",
        "risk_analysis": "",
        "technical_analysis": "",
        "final_recommendation": "",
        "messages": [HumanMessage(content=user_query)]
    }
)

# Print the final result
print("\n" + "="*80)
print("FINAL INVESTMENT STRATEGY")
print("="*80)
print(final_state["final_recommendation"])
print("\n" + "="*80)

GLM LLM (ZhipuAI via OpenAI Client) Initialized: LIVE
DeepSeek LLM (LangChain) Initialized: LIVE
Kraken Exchange (CCXT) Initialized: LIVE

RUNNING MULTI-AGENT CRYPTO ANALYSIS
Query: What is the current investment thesis for Ethereum (ETH) based on live Kraken data? Provide a Buy/Hold/Sell recommendation.
Agents Status: GLM-4.6 (Live), DeepSeek (Live)
Data Source: Kraken Exchange (via CCXT)


FINAL INVESTMENT STRATEGY
### Investment Recommendation: **Buy**

#### Justification Based on Synthesis of Qualitative and Quantitative Analysis

The current investment thesis for Ethereum (ETH) is **bullish in the short term**, driven by strong technical momentum and optimistic market sentiment, as reflected in the live Kraken data. However, this outlook is tempered by identifiable risks that require careful management. Here's a balanced assessment:

- **Technical Momentum (Quantitative Analysis):** ETH is trading at $4,302.06, near the 24-hour high of $4,325.62, with a +3.74% price change indicat

## NEW

In [7]:
# Install required packages
!pip install langchain-core langgraph langchain-zhipuai langchain-openai ccxt -q

import operator
import os
import logging
import ccxt
from typing import TypedDict, Annotated, List
from langchain_core.messages import BaseMessage, HumanMessage, SystemMessage, AIMessage
from langchain_core.tools import tool
from langgraph.graph import StateGraph, END
from langchain_openai import ChatOpenAI
from openai import OpenAI

# Set up basic logging
logger = logging.getLogger(__name__)

# --- Environment and LLM Initialization ---

glm_client = None
deepseek_llm = None
kraken_exchange = None
GLM_STATUS = "Mock (Key Missing)"

# API Key Loading from Userdata
try:
    from google.colab import userdata
    ZAI_KEY_API = userdata.get('ZAI_KEY')
    DEEPSEEK_API_KEY = userdata.get('DEEPSEEK_API_KEY')
    KRAKEN_API_KEY = userdata.get('KRAKEN')
    KRAKEN_SECRET = userdata.get('KRAKEN_SECRET')
except Exception as e:
    print(f"Error loading Colab Userdata: {e}. All keys initialized as None.")
    ZAI_KEY_API = None
    DEEPSEEK_API_KEY = None
    KRAKEN_API_KEY = None
    KRAKEN_SECRET = None

# Initialize GLM LLM Client (Direct OpenAI Client for Zhipu AI)
if ZAI_KEY_API:
    try:
        glm_client = OpenAI(
            api_key=ZAI_KEY_API,
            base_url="https://open.bigmodel.cn/api/paas/v4/"
        )
        GLM_STATUS = "Live"
        print("GLM LLM (ZhipuAI via OpenAI Client) Initialized: LIVE")
    except Exception as e:
        print(f"GLM LLM (ZhipuAI) FAILED TO CONNECT ({e}). Setting to None.")
        glm_client = None
        GLM_STATUS = "Mock (API Failed)"
else:
    print(f"GLM LLM (ZhipuAI) Initialized: {GLM_STATUS}")

# Initialize DeepSeek LLM
if DEEPSEEK_API_KEY:
    try:
        deepseek_llm = ChatOpenAI(
            api_key=DEEPSEEK_API_KEY,
            base_url="https://api.deepseek.com/v1",
            model="deepseek-reasoner",
            temperature=0.0
        )
        print("DeepSeek LLM (LangChain) Initialized: LIVE")
    except Exception as e:
        print(f"DeepSeek LLM FAILED TO CONNECT ({e}). Setting to None.")
        deepseek_llm = None
else:
    print("DeepSeek LLM NOT INITIALIZED (Missing DEEPSEEK_API_KEY).")

# Initialize CCXT/Kraken Exchange - USE REAL DATA
try:
    kraken_exchange = ccxt.kraken({
        'apiKey': KRAKEN_API_KEY if KRAKEN_API_KEY else 'test_key',
        'secret': KRAKEN_SECRET if KRAKEN_SECRET else 'test_secret',
        'enableRateLimit': True,
    })
    print("Kraken Exchange (CCXT) Initialized: LIVE - USING REAL MARKET DATA")
except Exception as e:
    print(f"Error initializing Kraken CCXT: {e}. Will use public endpoints.")
    kraken_exchange = ccxt.kraken({
        'enableRateLimit': True,
    })

# --- Define Tools (Using REAL CCXT for Crypto Data) ---

@tool
def crypto_price_tool(ticker: str) -> str:
    """Fetches the current REAL price and indicators using the Kraken API via CCXT."""
    try:
        symbol = f"{ticker.upper()}/USD"
        ticker_data = kraken_exchange.fetch_ticker(symbol)

        last_price = ticker_data['last']
        percent_change = ticker_data['percentage'] * 100  # Convert to percentage
        high_24h = ticker_data['high']
        low_24h = ticker_data['low']
        base_volume = ticker_data['baseVolume']
        quote_volume = ticker_data['quoteVolume']

        # Calculate additional metrics
        price_range = high_24h - low_24h
        range_percentage = (price_range / low_24h) * 100
        mid_range = (high_24h + low_24h) / 2

        # Determine trend based on position in range
        range_position = ((last_price - low_24h) / price_range) * 100
        if range_position >= 60:
            trend = "STRONGLY BULLISH (trading in upper 40% of range)"
        elif range_position >= 40:
            trend = "BULLISH (trading in middle-upper range)"
        elif range_position >= 20:
            trend = "BEARISH (trading in middle-lower range)"
        else:
            trend = "STRONGLY BEARISH (trading in lower 20% of range)"

        return (
            f"🔴 LIVE KRAKEN MARKET DATA for {symbol}:\n"
            f"💰 Current Price: ${last_price:.2f}\n"
            f"📊 24h Change: {percent_change:+.2f}%\n"
            f"📈 24h High: ${high_24h:.2f}\n"
            f"📉 24h Low: ${low_24h:.2f}\n"
            f"🎯 Price Range: ${price_range:.2f} ({range_percentage:.1f}% of low)\n"
            f"📊 Range Position: {range_position:.1f}% from low\n"
            f"🚀 Market Trend: {trend}\n"
            f"💧 Volume (Base): {base_volume:.2f} {ticker.upper()}\n"
            f"💧 Volume (Quote): ${quote_volume:.2f}\n"
            f"⚡ Resistance Level: ${high_24h:.2f}\n"
            f"🛡️ Support Level: ${low_24h:.2f}"
        )
    except ccxt.BaseError as e:
        return f"❌ CCXT Error fetching {ticker}: {e}"
    except Exception as e:
        return f"❌ Unexpected error: {e}"

# --- Define Graph State (Shared Memory) ---

class AgentState(TypedDict):
    """The state shared between nodes in LangGraph."""
    query: str
    tool_output: str
    risk_analysis: str
    technical_analysis: str
    final_recommendation: str
    messages: Annotated[List[BaseMessage], operator.add]

# --- Agent Nodes ---

def data_fetcher(state: AgentState) -> AgentState:
    """Node 1: Fetches REAL raw data from Kraken."""
    query = state["query"]
    # Extract ticker from query
    if "ethereum" in query.lower() or "eth" in query.lower():
        ticker = "ETH"
    elif "bitcoin" in query.lower() or "btc" in query.lower():
        ticker = "BTC"
    else:
        ticker = "ETH"  # default

    data = crypto_price_tool.invoke({"ticker": ticker})
    return {"tool_output": data, "messages": [AIMessage(content=f"Data Fetcher completed. REAL Market Data: {data}")]}

def risk_analyst_glm(state: AgentState) -> AgentState:
    """Node 2: GLM-4 (Qualitative/Risk) performs analysis using REAL data."""
    if glm_client is None:
        return {"risk_analysis": f"GLM-4 MOCK: Client not connected. Cannot perform qualitative analysis."}

    system_prompt = (
        "You are the GLM-4 Qualitative Risk Analyst. Analyze the REAL market data including:\n"
        "- 24h Price Change percentage\n"
        "- Current price position within the 24h range\n"
        "- Volume metrics\n"
        "- Market trend classification\n\n"
        "Identify the *implied market sentiment* and suggest potential *macro risks*. "
        "Focus on: regulatory risks, market sentiment shifts, liquidity concerns, "
        "and broader economic factors that could impact this cryptocurrency. "
        "Provide specific risk levels (Low/Medium/High) for current market conditions."
    )

    prompt = f"{system_prompt}\n\nREAL MARKET DATA:\n{state['tool_output']}"

    try:
        response = glm_client.chat.completions.create(
            model="glm-4",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": prompt}
            ],
            temperature=0.1
        )
        analysis_content = response.choices[0].message.content

    except Exception as e:
        analysis_content = f"GLM-4 API FAILED during invocation: {e}"

    return {"risk_analysis": analysis_content, "messages": [AIMessage(content=f"Risk Analyst (GLM) completed: {analysis_content}")]}

def technical_analyst_deepseek(state: AgentState) -> AgentState:
    """Node 3: DeepSeek (Quantitative) performs analysis using REAL data."""
    if deepseek_llm is None:
        return {"technical_analysis": "DeepSeek Mock: LLM failed to initialize. Cannot perform quantitative analysis."}

    system_prompt = (
        "You are the DeepSeek Quantitative Analyst, specializing in REAL price action analysis. "
        "Analyze the following REAL market metrics:\n"
        "- Current Price and 24h Change\n"
        "- 24h Range (High/Low) and current position within range\n"
        "- Volume data (both base and quote)\n"
        "- Resistance and Support levels\n\n"
        "Based on the REAL data, provide:\n"
        "1. Immediate price outlook (Bullish/Bearish/Neutral)\n"
        "2. Specific entry price levels\n"
        "3. Stop-loss levels with risk calculation\n"
        "4. Take-profit targets\n"
        "5. Risk-reward ratio assessment\n"
        "Use ACTUAL price levels from the data for all calculations."
    )
    prompt = f"{system_prompt}\n\nREAL MARKET DATA:\n{state['tool_output']}"

    # LangChain invocation
    response = deepseek_llm.invoke([HumanMessage(content=prompt)])

    return {"technical_analysis": response.content, "messages": [AIMessage(content=f"Technical Analyst (DeepSeek) completed: {response.content}")]}

def final_synthesizer(state: AgentState) -> AgentState:
    """Node 4: Synthesizes both analyses into final recommendation using REAL data."""
    llm_for_synthesis = deepseek_llm if deepseek_llm else None

    system_prompt = (
        "You are the Lead Investment Strategist. Synthesize the Qualitative Risk Analysis (GLM-4) and "
        "Quantitative Technical Analysis (DeepSeek) below to provide a single, actionable investment recommendation.\n\n"
        "CRITICAL: All analyses are based on REAL, LIVE market data from Kraken exchange.\n\n"
        "Provide:\n"
        "1. Clear Buy/Hold/Sell recommendation\n"
        "2. Specific price targets from REAL data\n"
        "3. Risk management strategy using ACTUAL support/resistance levels\n"
        "4. Position sizing suggestion\n"
        "5. Time horizon for the trade\n\n"
        "Base ALL calculations on the REAL market data provided."
    )

    full_analysis = (
        f"=== REAL MARKET DATA ===\n{state['tool_output']}\n\n"
        f"=== QUALITATIVE RISK ANALYSIS (GLM-4) ===\n{state['risk_analysis']}\n\n"
        f"=== QUANTITATIVE TECHNICAL ANALYSIS (DeepSeek) ===\n{state['technical_analysis']}"
    )

    if llm_for_synthesis:
        # Use DeepSeek for synthesis
        response = llm_for_synthesis.invoke([
            SystemMessage(content=system_prompt),
            HumanMessage(content=full_analysis)
        ])
        final_recommendation = response.content
    elif glm_client:
        # Fallback to GLM
        try:
            response = glm_client.chat.completions.create(
                model="glm-4",
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": full_analysis}
                ]
            )
            final_recommendation = response.choices[0].message.content
        except:
            final_recommendation = "Synthesis Error: Both LLMs failed."
    else:
        final_recommendation = "Synthesis Error: No LLMs available."

    return {"final_recommendation": final_recommendation, "messages": [AIMessage(content=f"Final Strategist completed using REAL data.")]}

# --- Define and Run the Workflow Graph (LangGraph) ---

workflow = StateGraph(AgentState)

# Add the nodes
workflow.add_node("data_fetcher", data_fetcher)
workflow.add_node("risk_analyst", risk_analyst_glm)
workflow.add_node("technical_analyst", technical_analyst_deepseek)
workflow.add_node("final_synthesizer", final_synthesizer)

# Define the flow (Edges)
workflow.set_entry_point("data_fetcher")
workflow.add_edge("data_fetcher", "risk_analyst")
workflow.add_edge("data_fetcher", "technical_analyst")

# Fan-in
workflow.add_edge("risk_analyst", "final_synthesizer")
workflow.add_edge("technical_analyst", "final_synthesizer")

# End the graph
workflow.add_edge("final_synthesizer", END)

# Compile the graph
app = workflow.compile()

# --- Execute with REAL Data ---

user_query = "What is the current investment thesis for Ethereum (ETH) based on LIVE Kraken data? Provide a Buy/Hold/Sell recommendation with specific price levels."

print(f"\\n{'='*80}")
print("🚀 MULTI-AGENT CRYPTO ANALYSIS WITH REAL MARKET DATA")
print(f"{'='*80}")
print(f"📋 Query: {user_query}")
print(f"🤖 Agents Status: GLM-4 ({GLM_STATUS}), DeepSeek ({'Live' if deepseek_llm else 'Mock'})")
print(f"📊 Data Source: Kraken Exchange (CCXT) - USING REAL MARKET DATA")
print(f"{'='*80}\\n")

final_state = app.invoke(
    {
        "query": user_query,
        "tool_output": "",
        "risk_analysis": "",
        "technical_analysis": "",
        "final_recommendation": "",
        "messages": [HumanMessage(content=user_query)]
    }
)

# Print the final result
print("\\n" + "="*80)
print("🎯 FINAL INVESTMENT STRATEGY (BASED ON REAL DATA)")
print("="*80)
print(final_state["final_recommendation"])
print("\\n" + "="*80)

GLM LLM (ZhipuAI via OpenAI Client) Initialized: LIVE
DeepSeek LLM (LangChain) Initialized: LIVE
Kraken Exchange (CCXT) Initialized: LIVE - USING REAL MARKET DATA
\n================================================================================
🚀 MULTI-AGENT CRYPTO ANALYSIS WITH REAL MARKET DATA
📋 Query: What is the current investment thesis for Ethereum (ETH) based on LIVE Kraken data? Provide a Buy/Hold/Sell recommendation with specific price levels.
🤖 Agents Status: GLM-4 (Live), DeepSeek (Live)
📊 Data Source: Kraken Exchange (CCXT) - USING REAL MARKET DATA
================================================================================\n
\n================================================================================
🎯 FINAL INVESTMENT STRATEGY (BASED ON REAL DATA)
Based on the synthesis of available data, here is my actionable investment recommendation:

## 🟢 INVESTMENT RECOMMENDATION: CONDITIONAL BUY

**Primary Recommendation:** Execute LONG position using the Conservative Ent